In [1]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

c:\users\admin\appdata\local\programs\python\python37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\admin\appdata\local\programs\python\python37\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
c:\users\admin\appdata\local\programs\python\python37\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file('Tensorflow/workspace/training_demo/models/my-fast-rcnn-resnet-model/pipeline.config')
fast_rcnn_resnet = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=fast_rcnn_resnet)
ckpt.restore(os.path.join('Tensorflow/workspace/training_demo/models/my-fast-rcnn-resnet-model', 'ckpt-11')).expect_partial() #latest model

@tf.function
def detect_fn(image):
    image, shapes = fast_rcnn_resnet.preprocess(image)
    prediction_dict = fast_rcnn_resnet.predict(image, shapes)
    detections = fast_rcnn_resnet.postprocess(prediction_dict, shapes)
    return detections

In [11]:
import cv2 
import numpy as np
from win32com.client import Dispatch
category_index = label_map_util.create_category_index_from_labelmap('Tensorflow/workspace/training_demo/Annotations/label_map.pbtxt')

cap = cv2.VideoCapture(0)


while True: 
    (ret, frame) = cap.read()
    frame=cv2.flip(frame,1,1) #Flip to act as a mirror
    image_np = np.array(frame)#to convert the frame to an array
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.8,
                agnostic_mode=False)

    
    cv2.imshow('ASL to Text',  cv2.resize(image_np_with_detections, (900,700)))
    
    if detections['detection_scores'][0]*100>80:
        speech=(detections['detection_classes']+1)[0]
        speak = Dispatch("SAPI.SpVoice").Speak
        speak(category_index[speech]['name'])
        print(category_index[speech]['name'])
        
    key=cv2.waitKey(10)
    
    if key == ord('q') or key== ord('Q'):
        break
        
cap.release()
cv2.destroyAllWindows()





Hello
Hello
Hello
Help
Help
Hurt
Hurt
Hurt
LoveYou
LoveYou
One
One
Phone
Please
Please
ThankYou
ThankYou
Yes
Yes
No
No
